# Yes24 크롤링

## 개요

1. 구글 홈페이지의 임의의 class attribute로 html 컨텐츠 가져오기
2. Yes24 베스트셀러 컨텐츠 가져오기
3. 최대 3페이지까지 가지고 오기


In [43]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
import re
from datetime import datetime

## webdriver manager 불러오기
ChromeDriverManager().install()

## 자동화 크롬 실행하기
browser = webdriver.Chrome()
browser.get("https://google.com")

여기까지 하면 아래와 같이 새 크롬 탭이 열리는 것을 볼 수 있다:

![resource/Screenshot 2024-04-08 at 21.47.50.png](./resource/Screenshot%202024-04-08%20at%2021.47.50.png)

그럼 이제 구글 페이지에서 아무 컴포넌트의 텍스트를 불러와보자. 하단에 있는 "대한민국" 버튼을 불러보자.


In [44]:
## google 페이지에서 "대한민국" 가져오기

class_name = "uU7dJb"

element = browser.find_element(By.CLASS_NAME, class_name)

print(element.text)

대한민국


## 크롤링을 사용하여 Yes24 베스트 셀러 데이터 수집하기

1. yes24 베스트셀러 항목 보기로 간다.
2. 모든 베스트셀러 항목들의 상세 페이지를 순회한다.
3. 3페이지까지 반복한다.

### 수집할 정보들

- 책 제목 (title)
- 저자 (author)
- 출판사 (publisher)
- 출판일 (publishing)
- 평점 (rating)
- 리뷰 (review)
- 판매지수 (sales)
- 가격 (price)
- 국내도서랭킹 (ranking)
- 국내도서TOP100 (ranking_weeks)


In [46]:
link_list = []

## 1 페이지의 링크 데이터 수집

# elements = browser.find_elements(By.CLASS_NAME, class_name)

# for element in elements:
#     print(element.get_attribute("href"))

## 1 ~ 3 페이지의 링크 데이터 수집
for i in range(1, 4):
    url = f"https://www.yes24.com/Product/Category/BestSeller?pageNumber={i}&pageSize=24"

    browser.get(url)

    elements = browser.find_elements(By.CLASS_NAME, "gd_name")
    for element in elements:
        link_list.append(element.get_attribute("href"))
        print(element.text, end=": \n")
        print(element.get_attribute("href"), "\n")

    sleep(0.2)

불변의 법칙: 
https://www.yes24.com/Product/Goods/124999476 

나를 소모하지 않는 현명한 태도에 관하여: 
https://www.yes24.com/Product/Goods/125295101 

일류의 조건: 
https://www.yes24.com/Product/Goods/125491624 

이상한 과자 가게 전천당 20: 
https://www.yes24.com/Product/Goods/125871480 

꽃길이 따로 있나, 내 삶이 꽃인 것을: 
https://www.yes24.com/Product/Goods/125576919 

나는 메트로폴리탄 미술관의 경비원입니다: 
https://www.yes24.com/Product/Goods/123675187 

세이노의 가르침: 
https://www.yes24.com/Product/Goods/117014613 

마흔에 읽는 쇼펜하우어: 
https://www.yes24.com/Product/Goods/122120495 

에그박사 12: 
https://www.yes24.com/Product/Goods/125752452 

스킵과 로퍼 8: 
https://www.yes24.com/Product/Goods/125905738 

나는 나의 스무 살을 가장 존중한다: 
https://www.yes24.com/Product/Goods/124926484 

삼체 1~3 세트: 
https://www.yes24.com/Product/Goods/90926731 

여행의 이유: 
https://www.yes24.com/Product/Goods/125905365 

돈의 심리학 (30만 부 기념 스페셜 에디션): 
https://www.yes24.com/Product/Goods/96547408 

2024 제15회 젊은작가상 수상작품집: 
https://www.yes24.com/Product/Goods/125709308 

양수인간: 
https://www.yes24.com/P

## Chapter 5-32. 예스24 베스트셀러 데이터 수집 후 데이터 분석하기


In [33]:
link_list

['https://www.yes24.com/Product/Goods/124999476',
 'https://www.yes24.com/Product/Goods/125295101',
 'https://www.yes24.com/Product/Goods/125491624',
 'https://www.yes24.com/Product/Goods/125871480',
 'https://www.yes24.com/Product/Goods/125576919',
 'https://www.yes24.com/Product/Goods/123675187',
 'https://www.yes24.com/Product/Goods/117014613',
 'https://www.yes24.com/Product/Goods/122120495',
 'https://www.yes24.com/Product/Goods/125752452',
 'https://www.yes24.com/Product/Goods/125905738',
 'https://www.yes24.com/Product/Goods/124926484',
 'https://www.yes24.com/Product/Goods/90926731',
 'https://www.yes24.com/Product/Goods/125905365',
 'https://www.yes24.com/Product/Goods/96547408',
 'https://www.yes24.com/Product/Goods/125709308',
 'https://www.yes24.com/Product/Goods/125453202',
 'https://www.yes24.com/Product/Goods/125384202',
 'https://www.yes24.com/Product/Goods/123400303',
 'https://www.yes24.com/Product/Goods/125184439',
 'https://www.yes24.com/Product/Goods/124043812',
 '

In [49]:
books = []

for link in link_list:
    browser.get(link)
    ranks = (
        browser.find_element(By.CLASS_NAME, "gd_best")
        .find_element(By.TAG_NAME, "dd")
        .text.split(" | ")
    )
    publishing = browser.find_element(By.CLASS_NAME, "gd_date").text
    match = re.search(r"(\d+)년 (\d+)월 (\d+)일", publishing)
    if match:
        year, month, day = (int(x) for x in match.groups())
        date_obj = datetime(year, month, day)
        publishing = date_obj.strftime("%Y-%m-%d")
    else:
        publishing = None

    rating = browser.find_element(By.CLASS_NAME, "yes_b").text
    if len(rating) > 0:
        rating = float(rating)
    else:
        rating = None

    review = browser.find_element(By.CLASS_NAME, "gd_reviewCount").text.replace(",", "")
    del match
    match = re.search(r"\d+", review)
    if match:
        review = int(match.group())
    else:
        review = None

    book = {
        "title": browser.find_element(By.CLASS_NAME, "gd_name").text,
        "auth": browser.find_element(By.CLASS_NAME, "gd_auth").text,
        "publisher": browser.find_element(By.CLASS_NAME, "gd_pub").text,
        "publishing": publishing,
        "rating": rating,
        "review": review,
        "sales": int(
            browser.find_element(By.CLASS_NAME, "gd_sellNum").text.split()[2].replace(",", "")
        ),
        "price": int(browser.find_element(By.CLASS_NAME, "yes_m").text[:-1].replace(",", "")),
        "ranking": int(ranks[0][-2:-1]),
        "ranking_weeks": int(ranks[1][-2:-1]) if len(ranks) > 1 else None,
    }

    print(book)
    books.append(book)

    sleep(0.1)

{'title': '불변의 법칙', 'auth': '모건 하우절 저/이수경 역', 'publisher': '서삼독', 'publishing': '2024-02-28', 'rating': 9.8, 'review': 120, 'sales': 465678, 'price': 25000, 'ranking': 1, 'ranking_weeks': 5}
{'title': '나를 소모하지 않는 현명한 태도에 관하여', 'auth': '마티아스 뇔케 저/이미옥 역', 'publisher': '퍼스트펭귄', 'publishing': '2024-03-10', 'rating': 9.7, 'review': 50, 'sales': 249369, 'price': 17800, 'ranking': 2, 'ranking_weeks': 4}
{'title': '일류의 조건', 'auth': '사이토 다카시 저/정현 역', 'publisher': '필름', 'publishing': '2024-03-20', 'rating': 9.8, 'review': 195, 'sales': 246339, 'price': 19800, 'ranking': 3, 'ranking_weeks': 4}
{'title': '이상한 과자 가게 전천당 20', 'auth': '히로시마 레이코 글/쟈쟈 그림/김정화 역', 'publisher': '길벗스쿨', 'publishing': '2024-04-22', 'rating': 10.0, 'review': 3, 'sales': 56310, 'price': 13000, 'ranking': 4, 'ranking_weeks': 1}
{'title': '꽃길이 따로 있나, 내 삶이 꽃인 것을', 'auth': '오평선 저', 'publisher': '포레스트북스', 'publishing': '2024-03-22', 'rating': 9.8, 'review': 43, 'sales': 110733, 'price': 16800, 'ranking': 5, 'ranking_weeks': 2}
{'t

## pymysql과 연동하기


In [53]:
import pymysql
from pymysql.cursors import DictCursor

with pymysql.connect(
    host="localhost",
    user="root",
    password="oz-password",
    db="yes24",
    charset="utf8mb4",
    cursorclass=DictCursor,
) as conn:
    with conn.cursor() as cur:
        for book in books:
            sql = """INSERT INTO Books(
                title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

            cur.execute(sql, tuple(book.values()))

        conn.commit()